In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
base_dir = Path("/Users/sylvi/topo_data/topostats_2/datasets/topology-plasmids")
results_dir = base_dir / "output_old_catsnet"
assert results_dir.exists()

file_allstats = results_dir / "all_statistics.csv"
assert file_allstats.exists()
df_allstats = pd.read_csv(file_allstats)
print(df_allstats.columns)

# convert to nm
df_allstats["area"] = df_allstats["area"] * 1e18
df_allstats["total_contour_length"] = df_allstats["total_contour_length"] * 1e9

# print the writhe_string column unique values
print(df_allstats["writhe_string"].unique())


def calculate_num_char_in_string(input_string: str, character: str) -> int:
    """Calculate the number of occurrences of a specific character in a string."""
    # check if nan
    if pd.isna(input_string):
        return 0
    return input_string.count(character)


df_allstats["num_plusses"] = df_allstats["writhe_string"].apply(calculate_num_char_in_string, character="+")
df_allstats["num_minuses"] = df_allstats["writhe_string"].apply(calculate_num_char_in_string, character="-")
df_allstats["num_plusses_or_minuses"] = df_allstats["num_plusses"] + df_allstats["num_minuses"]

In [ ]:
# plot violin of length based on basename
sns.violinplot(data=df_allstats, x="basename", y="total_contour_length", inner="point")
plt.ylabel("Contour Length (nm)")
plt.show()

# plot num of plusses or minuses based on basename
sns.violinplot(data=df_allstats, x="basename", y="num_plusses_or_minuses", inner="point")
plt.ylabel("Number of crossings")
plt.show()

#